<a href="https://colab.research.google.com/github/aysegulYalcinkaya/AIDI-1100-Project1/blob/main/bookstore_scrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from requests.models import HTTPError
from requests.exceptions import ConnectTimeout
import requests
from bs4 import BeautifulSoup
import sqlite3

In [4]:
def create_database():
  conn = sqlite3.connect('bookstore.db')
 
  conn.execute('''
  CREATE TABLE books_scraped (
    id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    Title NVARCHAR NOT NULL ,
    Price REAL NOT NULL 
  );''')
  
  conn.commit()
  
  print("books_scraped table created");
  

In [17]:
def insert_book(conn,book):
  sql='INSERT INTO books_scraped (title,price) VALUES(?,?)'
  cur = conn.cursor()
  cur.execute(sql, book)
  conn.commit()
  return cur.lastrowid

In [26]:
def getPage(page_number):
  try:
    url = "https://books.toscrape.com/catalogue/page-"+str(page_number)+".html"
    payload={}
    files={}
    headers = {}

    response = requests.request("GET", url, headers=headers, data=payload, files=files)
    response.encoding='utf-8'
    return response.text
  except ConnectionError:
    print("Connection error occured")
  except (ConnectTimeout, TimeoutError) as timeout:
    print("Connection timeout")
  except HTTPError:
    print("HTTP error occurred")
 

In [35]:
create_database()

books_scraped table created


In [36]:
from sqlite3.dbapi2 import OperationalError
page=1
next=None
try:
  conn = sqlite3.connect('bookstore.db')

  while True:
    html=getPage(page)
  
    if html!=None:
      soup = BeautifulSoup(html, "lxml")
      next = soup.find("li",class_="next")
      list=soup.find("ol", class_="row")
      books=list.find_all("li")


      for book in books:
        if book.find("h3").find("a"):
          title=book.find("h3").find("a")["title"]
        else:
          title=""
        if book.find("div",class_="product_price").find("p",class_="price_color"):
          price=book.find("div",class_="product_price").find("p",class_="price_color").get_text()
        else:
          price=None
      
        insert_book(conn,(title,price))
      
      page+=1

      if not next:
        break
    else:
      break

  conn.close()
except OperationalError as ex:
  print(ex)
  

In [45]:
conn = sqlite3.connect('bookstore.db')
cursor = conn.cursor()
data=cursor.execute('''SELECT * FROM books_scraped''')
for row in data:
  pass
  #print(row)

import pandas as pd

df = pd.read_sql_query('''SELECT * FROM books_scraped''',conn)
df["date"]="2023-04-06"
df.to_csv("books.csv",index=False)
# Closing the connection
conn.close()